<a href="https://colab.research.google.com/github/Andchh/APACountingSort/blob/master/WheresWally/01_WheresWally_Anderson_Fraz%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neste primeiro teste decidi usar OpenCV pela praticidade que o mesmo trás em problemas que é necessário encontrar objetos ou pessoas na foto.

Foi utilizado uma técnica de Feature Matching conhecida como ORB (Oriented and Rotated BRIEF), que é capaz de encontrar pontos chave em imagens e comparar com a imagem de referência.

#Detectando com ORB

In [6]:
import numpy as np
import cv2
import glob
import os
import pandas as pd
import csv

imgname = "/content/01-WheresWally/ReferenceData/wally.jpg"          # imagem a ser procurada


In [2]:
#baixando e extraindo o dataset fornecido para o teste
!gdown --id 1s1wts4jmqOIEHYU7iqK7nSkgG3eoZscY
!unzip -q /content/01-WheresWally.zip

Downloading...
From: https://drive.google.com/uc?id=1s1wts4jmqOIEHYU7iqK7nSkgG3eoZscY
To: /content/01-WheresWally.zip
49.5MB [00:00, 60.6MB/s]


In [7]:
#iterando pela pasta de testes e salvando 
for imagePath in glob.glob('/content/01-WheresWally/TestSet' + "/*.jpg"):
  #print(os.path.basename(imagePath))
  imgname2 = imagePath
  MIN_MATCH_COUNT = 4

  ## criando ORB e BF (using HAMMING)
  orb = cv2.ORB_create()
  img1 = cv2.imread(imgname)
  img2 = cv2.imread(imgname2)
  #passando pra cinza
  gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
  gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

  
  #Encontrando os keypoints e descriptors com ORB
  kpts1, descs1 = orb.detectAndCompute(gray1,None)
  kpts2, descs2 = orb.detectAndCompute(gray2,None)

  #comparando descriptors e organizando eles por ordem de distância
  bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
  matches = bf.match(descs1, descs2)
  dmatches = sorted(matches, key = lambda x:x.distance)

  #extraindo keypoints
  src_pts  = np.float32([kpts1[m.queryIdx].pt for m in dmatches]).reshape(-1,1,2)
  dst_pts  = np.float32([kpts2[m.trainIdx].pt for m in dmatches]).reshape(-1,1,2)

  #encontrando matriz de homografia e fazendo transformação de perspectiva
  M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
  h,w = img1.shape[:2]
  pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
  dst = cv2.perspectiveTransform(pts,M)

  #pegando as coordenadas de wally na imagem
  coord = dst.reshape(4,2)
  coord = np.int32(coord)

  bottom_right = coord[0]
  top_right = coord[1]
  top_left = coord[2]
  bottom_left = coord[3]

  #pegando os centroides
  xcenter = ((bottom_left[0] + bottom_right[0])/2)
  ycenter = (bottom_left[1] + top_left[1]) /2
  #print(xcenter, ycenter)
  csv_list = [os.path.basename(imagePath), xcenter, ycenter]
  #escrevendo o csv
  with open('output.csv', 'a', newline = '') as myfile:
    wr = csv.writer(myfile, quoting = csv.QUOTE_ALL)
    wr.writerow(csv_list)


